In [86]:
ljy_fp = r"C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\objdump-D\-dmanytest.asm"
GuGuJi_fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/newCFG/hw-memory/benchmarks/spec_example/spec2006_470.lbm/lbm_part.asm"
fp = ljy_fp

In [87]:
'''读指令, 指令对象化'''
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

for i in temp_stat:
    if i.is_ls:
        if i.ls_handle:
            print(i.tokens,i.ls_reg_target,i.ls_addr_mode,i.ls_addr_offset,i.local_offset)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

('4004b8', 'f947fe11', 'ldr', None, 'x17, [x16, #4088]') x16 AddrMode.ImmeOffset 4088 0
('4004d4', 'f9400211', 'ldr', None, 'x17, [x16]') x16 AddrMode.Base 0 0
('4004e4', 'f9400611', 'ldr', None, 'x17, [x16, #8]') x16 AddrMode.ImmeOffset 8 0
('4004f4', 'f9400a11', 'ldr', None, 'x17, [x16, #16]') x16 AddrMode.ImmeOffset 16 0
('400510', 'f94003e1', 'ldr', None, 'x1, [sp]') sp AddrMode.Base 0 0
('400548', 'f947ec00', 'ldr', None, 'x0, [x0, #4056]') x0 AddrMode.ImmeOffset 4056 0
('40057c', 'f947e821', 'ldr', None, 'x1, [x1, #4048]') x1 AddrMode.ImmeOffset 4048 0
('4005b8', 'f947f042', 'ldr', None, 'x2, [x2, #4064]') x2 AddrMode.ImmeOffset 4064 0
('4005d8', 'f9000bf3', 'str', None, 'x19, [sp, #16]') sp AddrMode.ImmeOffset 16 16
('4005e0', '39410260', 'ldrb', None, 'w0, [x19, #64]') x19 AddrMode.ImmeOffset 64 0
('4005f0', '39010260', 'strb', None, 'w0, [x19, #64]') x19 AddrMode.ImmeOffset 64 0
('4005f4', 'f9400bf3', 'ldr', None, 'x19, [sp, #16]') sp AddrMode.ImmeOffset 16 16
('40060c', 'b940

In [88]:
'''分proc'''
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output\\procedures.svg'

In [89]:
'''处理call_graph'''
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output\\call_graph.svg'

In [90]:
'''生成TCFG和loop信息'''
from newCFG.cfg import TCfg
import random

tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




In [91]:
'''设置node和edge上的的value'''

for l in tcfg.loops:
    l.back_edge.is_backEdge = True
for n in tcfg_nodes:
    n.set_rw_data()

temp_bool = True

while temp_bool:
    no_end = False

    for n in tcfg_nodes:
        n.set_rw_value()
        if not n.no_out:
            no_end = True
        temp_bool = no_end

for e in tcfg_edges:
    if e.is_backEdge:
        e.loop_value = random.randint(0,10)
        e.edge_value = 2

for e in tcfg_edges:
    print(e.src.name,e.dst.name,e.edge_value)

In [92]:
'''ls分析和读写特征'''

from newCFG.cfg import TCfg
from newCFG.loadstore import LSProc
from newCFG.rw_condition import RWProc, RWType

lsproc = LSProc(tcfg_nodes)

lds_table = lsproc.ls_table




for i in lds_table:
    i.final_addr
    i.local_offset

rwproc = RWProc(lds_table)

Global_Tolerant_value = 0
Global_Intolerant_value = 0

for rwu in rwproc.rw_table:
    #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
    if rwu.is_torrent == RWType.Global_Tolerant:
        Global_Tolerant_value += rwu.find_cycle
    elif rwu.is_torrent == RWType.Global_Intolerant:
        Global_Intolerant_value += rwu.find_cycle

print("全局的容错路径为：",Global_Tolerant_value)
print("全局的非容错路径为：",Global_Intolerant_value)   

全局的容错路径为： 13
全局的非容错路径为： 15


In [93]:
'''cache分析'''

#打印给cache分许需要的数据
for i in lds_table:
    print("指令",i.ins.tokens,"是否是SP",i.is_sp,"指令地址",i.ins.addr.val(),"访存地址",i.final_addr,"数据宽度",i.ins.ls_data_width)


指令 ('40060c', 'b9400000', 'ldr', None, 'w0, [x0]') 是否是SP False 指令地址 4195852 访存地址 4325420 数据宽度 32
指令 ('40061c', 'b9000001', 'str', None, 'w1, [x0]') 是否是SP False 指令地址 4195868 访存地址 4325420 数据宽度 32
指令 ('400628', 'bd400000', 'ldr', None, 's0, [x0]') 是否是SP False 指令地址 4195880 访存地址 4325416 数据宽度 32
指令 ('40064c', 'bd000000', 'str', None, 's0, [x0]') 是否是SP False 指令地址 4195916 访存地址 4325416 数据宽度 32
指令 ('400658', 'b9400001', 'ldr', None, 'w1, [x0]') 是否是SP False 指令地址 4195928 访存地址 4325420 数据宽度 32
指令 ('400664', 'b9400000', 'ldr', None, 'w0, [x0]') 是否是SP False 指令地址 4195940 访存地址 4325424 数据宽度 32
指令 ('400674', 'b9000001', 'str', None, 'w1, [x0]') 是否是SP False 指令地址 4195956 访存地址 4325488 数据宽度 32


In [94]:
from cache_analysis.read_segment import segmentReader
bread = segmentReader(r'C:\Users\13377\Desktop\hw-memory-master (1)\hw-memory-master\benchmarks\objdump-D\-D manytest.asm')
a=bread.getbss()
b=bread.getdata()
print(a)
print(b)
'''热度分析'''
def find_range(addr:int):
    for i in a:
        if int(addr)==i[2]:
            return i[3]
    for i in b:
        if int(addr)==i[2]:
            return i[3]
for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True
for n in tcfg.all_nodes:
    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
    print("{} {}".format(n.name, temp))
#将对应的指令添加到node的热度信息表中
for i in lds_table:
    if i.ins.name[0:1] == "l":
        #print(i.ins.tokens)
        i.node.loadlist.append((i.ins,i.final_addr))
    elif i.ins.name[0:1] == "s":
        i.node.storelist.append((i.ins,i.final_addr))

for node in tcfg_nodes:
    for i in node.loadlist:
        node.heat_ld_result[i[1],find_range(i[1])]=0
    for i in node.loadlist:
        node.heat_ld_result[i[1],find_range(i[1])]+=1
    for i in node.storelist:
        node.heat_st_result[i[1],find_range(i[1])]=0
    for i in node.storelist:
        node.heat_st_result[i[1],find_range(i[1])]+=1
for node in tcfg_nodes:
    print(node.name)
    print('load',node.heat_ld_result)
    print('store',node.heat_st_result)
for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True
    for j in l.all_nodes:
        print(j.name,j.heat_ld_result,j.heat_st_result)
        for key,value in j.heat_ld_result.items():
            if key in l.loop_ld_heat:
                l.loop_ld_heat[key]+=value
            else:
                l.loop_ld_heat[key]=value
        for key,value in j.heat_st_result.items():
            if key in l.loop_st_heat:
                l.loop_st_heat[key]+=value
            else:
                l.loop_st_heat[key]=value
    
    print(l.name,'ld_heat_analysis',l.loop_ld_heat,'st_heat_analysis',l.loop_st_heat)



[['0000000000420040', 'completed.0', 4325440, 4325448], ['0000000000420048', 'test', 4325448, 4325488], ['0000000000420070', 'n', 4325488, 4325492]]
[['0000000000420018', '__data_start', 4325400, 4325408], ['0000000000420020', '__dso_handle', 4325408, 4325416], ['0000000000420028', 'i', 4325416, 4325420], ['000000000042002c', 'j', 4325420, 4325424], ['0000000000420030', 'k', 4325424, 4325432], ['0000000000420038', 'h', 4325432, 4325436]]
n0 None
n0
load {(4325420, 4325424): 2, (4325416, 4325420): 1, (4325424, 4325432): 1}
store {(4325420, 4325424): 1, (4325416, 4325420): 1, (4325488, 4325492): 1}
